In [1]:
!pip3 install pytesseract
!sudo apt install tesseract-ocr

import cv2
import imutils
from imutils import contours as cont
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import pytesseract
from PIL import Image
from pytesseract import image_to_string
from tensorflow import keras
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import numpy as np
import argparse
import imutils
import cv2
import tensorflow as tf
from google.colab import drive
from PIL import Image, ImageOps
from PIL import Image, ImageEnhance 
import pandas as pd
import os

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

drive.mount('/content/gdrive')

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=ce0ae77bbdaa6d59c438ac80cd9ed2b5a488874be476fd4aedf46021017126c8
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http:/

In [2]:
######################################
########## DIGIT OCR MODEL ###########
######################################

def predict_digit(output1,w1,w2,h1,h2):
  im = output1
  height = im.shape[0]
  width = im.shape[1]
  ratio = int(width/height *200)
  im = cv2.resize(im,(ratio,200))


  # RGB to grayscale conversion
  im_gray = im
  # grayscale to binary
  _, im_bw = cv2.threshold(im_gray, 100, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)

  # invert image
  im_bw = 255-im_bw

  # find contours
  cnts, hierarchy = cv2.findContours(im_bw, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

  # remove small components based on area
  if cnts is not None:
      for i in range(0, len(cnts)):
          a = cv2.contourArea(cnts[i])
          if a < 10:
              cv2.drawContours(im_bw, cnts, i, 0, cv2.FILLED)

  im_bw = 255-im_bw

  im_bw = cv2.GaussianBlur(im_bw , (5, 5), 0)

  cv2.imwrite('im_bw.jpg',im_bw)
  output= cv2.imread('im_bw.jpg')

  image = cv2.imread('im_bw.jpg')
  print("[INFO] loading handwriting OCR model...")
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blurred =cv2.bilateralFilter(gray,9,75,75)

  # perform edge detection, find contours in the edge map, and sort the
  # resulting contours from left-to-right
  edged = cv2.Canny(blurred, 50, 150)
  cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = sort_contours(cnts, method="left-to-right")[0]

  # initialize the list of contour bounding boxes and associated
  # characters that we'll be OCR'ing
  digits = []

  # loop over the contours
  for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)

    # filter out bounding boxes, ensuring they are neither too small
    # nor too large
    if ((w >= 3 and w <= 30  and h >= 50 +h1 and h <= 200 and y < 130 and y+h >=100) or
       (w > 30 and w <= 200  and h >= 50 + h2 and h <= 200 and y < 150 and y+h >=100)):
      # extract the character and threshold it to make the character
      # appear as *white* (foreground) on a *black* background, then
      # grab the width and height of the thresholded image
      roi = gray[y:y + h, x:x + w]
      thresh = cv2.threshold(roi, 0, 255,
        cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
      (tH, tW) = thresh.shape

      # if the width is greater than the height, resize along the
      # width dimension
      if tW > tH:
        thresh = imutils.resize(thresh, width=28)

      # otherwise, resize along the height
      else:
        thresh = imutils.resize(thresh, height=28)

      # re-grab the image dimensions (now that its been resized)
      # and then determine how much we need to pad the width and
      # height such that our image will be 28*28
      (tH, tW) = thresh.shape
      dX = int(max(0, 28 - tW) / 2.0)
      dY = int(max(0, 28 - tH) / 2.0)

      # pad the image and force 28 x 28 dimensions
      padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
        left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
        value=(0, 0, 0))
      padded = cv2.resize(padded, (28, 28))

      # prepare the padded image for classification via our
      # handwriting OCR model
      padded = padded.astype("float32") / 255.0
      padded = np.expand_dims(padded, axis=-1)

      # update our list of characters that will be OCR'd
      digits.append((padded, (x, y, w, h)))

  # extract the bounding box locations and padded characters
  boxes = [b[1] for b in digits]
  digits = np.array([c[0] for c in digits], dtype="float32")

  # OCR the characters using our handwriting recognition model
  preds = model.predict(digits)

  # define the list of label names
  labelNames = "0123456789"
  labelNames = [l for l in labelNames]
  string = ''
  # loop over the predictions and bounding box locations together
  for (pred, (x, y, w, h)) in zip(preds, boxes):
    # find the index of the label with the largest corresponding
    # probability, then extract the probability and label
    i = np.argmax(pred)
    prob = pred[i]
    if prob > 0.7:
      label = labelNames[i]
      # draw the prediction on the image
      print("[INFO] {} - {:.2f}%".format(label, prob * 100))
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(image, label, (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
      string += str(labelNames[i]) 
      #cv2_imshow(image)
  cv2_imshow(image)
  print('Outcome :' + string)
  return string



######################################
######### LETTER OCR MODEL ###########
######################################

word_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',
             6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',
             12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',
             18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}

def predict_letter(output1):

  im = output1
  height = im.shape[0]
  width = im.shape[1]
  ratio = int(width/height *200)
  im = cv2.resize(im,(ratio,200))

  # RGB to grayscale conversion
  im_gray = im

  # grayscale to binary
  _, im_bw = cv2.threshold(im_gray, 150, 250, cv2.THRESH_OTSU + cv2.THRESH_BINARY)

  # invert image
  im_bw = 255-im_bw

  # find contours
  cnts, hierarchy = cv2.findContours(im_bw, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

  # remove small components based on area
  if cnts is not None:
      for i in range(0, len(cnts)):
          a = cv2.contourArea(cnts[i])
          if a < 10:
              cv2.drawContours(im_bw, cnts, i, 0, cv2.FILLED)

  im_bw = 255-im_bw
  im_bw = cv2.GaussianBlur(im_bw , (5, 5), 0)
  cv2.imwrite('im_bw.jpg',im_bw)
  output= cv2.imread('im_bw.jpg')

  image = cv2.imread('im_bw.jpg')
  # load the handwriting OCR model
  print("[INFO] loading handwriting OCR model...")
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blurred =cv2.bilateralFilter(gray,9,35,35)

  # perform edge detection, find contours in the edge map, and sort the
  # resulting contours from left-to-right
  edged = cv2.Canny(blurred, 50, 150)
  cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = sort_contours(cnts, method="left-to-right")[0]

  # initialize the list of contour bounding boxes and associated
  # characters that we'll be OCR'ing
  chars = []
  string = ''
  # loop over the contours
  for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)

    # filter out bounding boxes, ensuring they are neither too small
    # nor too large
    if h > 30 and w > 5 and w < 400:
      # extract the character and threshold it to make the character
      # appear as *white* (foreground) on a *black* background, then
      # grab the width and height of the thresholded image
      roi = gray[y:y + h, x:x + w]

      img = cv2.resize(roi, (400,440))
      
      #repair_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
      #img = 255 - cv2.morphologyEx(255 - img, cv2.MORPH_CLOSE, repair_kernel, iterations=1)

      img_copy = cv2.GaussianBlur(img, (3,33), 0)
      img_gray = img_copy
      _, img_thresh = cv2.threshold(img_gray, 100, 250, cv2.THRESH_BINARY_INV)

      img_final = cv2.resize(img_thresh, (28,28))
      img_final =np.reshape(img_final, (1,28,28,1))


      img_pred = word_dict[np.argmax(model5.predict(img_final))]

      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(image ,img_pred, (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    
      string = str(img_pred)
      cv2_imshow(image)
      print('Outcome :' + string)
      break
  return string

#######################################
############# MATCHING ################
#######################################

def matching(ID,F,L):
  m = 0
  for index, row in df.iterrows():
    r = 0
    token = str(df.ID[index])


    d = abs(len(ID) - len(token))

    if len(token) > len(ID):
      for i in range(0,d+1):
        for j in range(0,len(ID)):
          if ID[j] == token[j+i]:
            r = r + 1
    else:
      if len(token) < len(ID):
        for i in range(0,d+1):
          for j in range(0,len(token)):
            if ID[j+i] == token[j]:
              r = r + 1
      else:
        for j in range(0,len(ID)):
          if ID[j] == token[j]:
            r = r + 1

    if F == df.firstname[index][0]:
      r = r +1

    if L == df.lastname[index][0]:
      r = r + 1
    
    if r > m:
      m = r
      ind = index

  
  #print(m)
  #print(ind)
  print("\nID :" ,str(df.ID[ind]))
  print("FirstName: " ,str(df.firstname[ind]))
  print("LastName: " ,str(df.lastname[ind]))
  return str(df.ID[ind]),str(df.firstname[ind]),str(df.lastname[ind])

############################################
################  MAIN  ####################
############################################

def Main(directory):
  df_new = pd.DataFrame(columns=['FILE NAME', 'FIRST NAME', 'LAST NAME',"STUDENT ID", 'TOTAL MARKS' ])
  for subdir, dirs, files in os.walk(directory):
    cnts = []
    i = 0

    for file in files:
      link = os.path.join(subdir, file)
      image = cv2.imread(F"{link}")
      print('\n# JPG LOEADED............')
      print("FILE NAME: " + str(file))


      image = cv2.resize(image,(1500,2100))
      result = image.copy()
      blur = cv2.GaussianBlur(image, (5,5), 0)
      gray = cv2.cvtColor(blur,cv2.COLOR_BGR2GRAY)
      thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

      # Remove horizontal lines
      horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30,1))
      remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
      cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      for c in cnts:
          cv2.drawContours(gray, [c], -1, (255,255,255), 3)

      # Remove vertical lines
      vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,30))
      remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
      cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      for c in cnts:
          cv2.drawContours(gray, [c], -1, (255,255,255), 3)

      # border widths; I set them all to 250
      color = [255, 255, 255]
      top, bottom, left, right = [250]*4
      result = cv2.copyMakeBorder(gray, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)



      ## cv2_imshow(result)


      # RGB to grayscale conversion
      #im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

      # grayscale to binary
      _, im_bw = cv2.threshold(result, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)

      # invert image
      im_bw = 255-im_bw

      # find contours
      cnts, hierarchy = cv2.findContours(im_bw, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

      # remove small components based on area
      if cnts is not None:
          for i in range(0, len(cnts)):
              a = cv2.contourArea(cnts[i])
              if a < 10:
                  cv2.drawContours(im_bw, cnts, i, 0, cv2.FILLED)

      im_bw = 255-im_bw
      cv2.imwrite('im_bw.jpg',im_bw)
      im_bw= cv2.imread('im_bw.jpg')


      # Load image, grayscale, Gaussian blur, adaptive threshold
      gray = cv2.cvtColor(im_bw, cv2.COLOR_BGR2GRAY)
      blur = cv2.GaussianBlur(gray, (9,9), 0)
      thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

      # Dilate to combine adjacent text contours
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
      dilate = cv2.dilate(thresh, kernel, iterations=4)

      # Find contours, highlight text areas, and extract ROIs
      cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      
      sub = 0 # this will have help to indentify type 5 later
      my_list = []
      for c in cnts:
          area = cv2.contourArea(c)
          if area > 500 and area < 7000:
            x,y,w,h = cv2.boundingRect(c)
            a = result[y-3:y + h, x: x + w]
            #cv2_imshow(a)
            b = pytesseract.image_to_string(a,config='--psm 6').lower()
            #print(b)
            if "venue" in b:  # this will have help to indentify type 5 later
              sub = 1 
            if (("q1" in b) or ("ql" in b) or ("qi" in b) or 
                  ("q2" in b) or ("q3" in b) or ("q4" in b) or 
                  ("q5" in b) or ("qs" in b) or ("q6" in b) or 
                  ("q7" in b) or ("q8" in b) or ("q9" in b) or
                  ("name" in b) or ("total" in b) or ("id" in b) or 
                  ("number" in b) or ("venue" in b)): 
              my_list.append(c)
              #cv2_imshow(a)
              #print(b)
                  

                    

### template structure type classify, get the family and first name

      for c in my_list:
        x,y,w,h = cv2.boundingRect(c)
        a = result[y -3 :y + h, x: x + w]
        b = pytesseract.image_to_string(a,config='--psm 6')
        b = b.lower()
        if "name" in b: 
          if "first" in pytesseract.image_to_string(result[y - 5 :y + h, x - 200: x + w],config='--psm 6').lower():
            if "given" in pytesseract.image_to_string(result[y - 5 :y + h, x - 200: x + w],config='--psm 6').lower():
              FirstName = result[y - 15 :y + h + 15, x + w + 100 : x + w + 450]
              cv2_imshow(FirstName)
              print("I")

            else:
              if "given" in pytesseract.image_to_string(result[y - 5 :y + h+30, x - 200: x + w],config='--psm 6').lower():
                print("\n")
                FirstName = result[y - 15 :y + h+30, x + w + 5: x + w +350]
                cv2_imshow(FirstName)
                print("II")
              
              else:
                print("\n")
                FirstName = result[y - 25 :y + h + 10 , x + w : x + w + 450]
                cv2_imshow(FirstName)
                print("III")
          
          else:
            if "given" in pytesseract.image_to_string(result[y - 15 :y + h, x - 150: x + w],config='--psm 6').lower():
              if "first" not in pytesseract.image_to_string(result[y - 40 :y + h, x - 100: x + w],config='--psm 6').lower():
                print("\n")
                FirstName = result[y - 35 :y + h +15 , x+w: x + w + 400]
                cv2_imshow(FirstName)
                print("IV")      


            else:
              if "family" in pytesseract.image_to_string(result[y - 5 :y + h, x - 200: x + w],config='--psm 6').lower():
                if "sur" in pytesseract.image_to_string(result[y - 5 :y + h, x - 200: x + w],config='--psm 6').lower():
                  print("\n")
                  SurName = result[y - 15 :y + h + 15, x + w + 100 : x + w + 450]
                  cv2_imshow(SurName)
                  t = 1
                  print("V")
                
                else:
                  if "sur" in pytesseract.image_to_string(result[y - 5 :y + h+40, x - 150: x + w],config='--psm 6').lower():
                    print("\n")
                    SurName = result[y - 15 :y + h+30, x + w +5: x + w + 350]
                    cv2_imshow(SurName)           
                    t = 2
                    print("VI")
                  
                  else:
                    if "family" in pytesseract.image_to_string(result[y - 15 :y + h, x - 200: x + w],config='--psm 6').lower():
                      if "block" in pytesseract.image_to_string(result[y - 15 :y + h, x : x + w + 150],config='--psm 6').lower():
                        print("\n")
                        SurName = result[y - 30 :y + h, x + w : x + w + 450]
                        cv2_imshow(SurName)
                        t = 3
                        print("VII")
              
                      else:
                        if "sur" not in pytesseract.image_to_string(result[y - 15 :y + h, x - 250: x + w],config='--psm 6').lower():
                          if sub == 0:
                            t = 4
                            SurName = result[y - 35 :y + h +10 , x + w +10: x + w +400]
                            cv2_imshow(SurName)   
                            print("X")
                          
                          if sub == 1:
                            t = 5
                            SurName = result[y - 20 :y + h +10 , x + w +10: x + w +400]
                            cv2_imshow(SurName)   
                            print("X - 1")
              else:
                if "family" and "sur" not in pytesseract.image_to_string(result[y - 5 :y + h, x - 130: x + w],config='--psm 6').lower():
                  print("\n")
                  SurName = result[y - 25 :y + h + 30, x + w + 5: x + w + 550]
                  cv2_imshow(SurName)
                  t = 0
                  print("XI")

      print('\nTYPE: ')
      print(t)

# getting ID

      for c in my_list:
        x,y,w,h = cv2.boundingRect(c)
        a = result[y -3 :y + h, x: x + w]
        b = pytesseract.image_to_string(a,config='--psm 6').lower()
        if "id" in b:
          if "student" in pytesseract.image_to_string(result[y - 5:y + h + 5, x - 150 : x + w]).lower():
            if "number" in pytesseract.image_to_string(result[y - 5:y + h + 5, x : x + w + 100]).lower():
              if t == 0:  
                ID = result[y - 25:y + h + 30, x + w + 100 : x + w + 450]
                cv2_imshow(ID)     
                print("XII")
                
            else:
              if t == 1: 
                ID = result[y -10 :y + h + 25, x + w : x + w + 500]
                cv2_imshow(ID)
                print("XIII")
              
              else:
                if t == 2:
                  ID = result[y - 45:y + h + 5, x + w +10 : x + w + 500]
                  cv2_imshow(ID)
                  print("XIII - 1")

                else:
                  if t == 4:
                    ID = result[y - 45:y + h + 5, x + w  : x + w + 500]
                    cv2_imshow(ID)
                    print("XIII - 2")
                
                  else:
                    ID = result[y - 35:y + h + 20, x + w + 100: x + w + 500]
                    cv2_imshow(ID)
                    print("XIII - 3")
              
        else:
          if "number" in b:
            if "student" in pytesseract.image_to_string(result[y - 5:y + h + 5, x - 100 : x + w]).lower():
              if "id" not in pytesseract.image_to_string(result[y - 5:y + h + 5, x - 100 : x + w]).lower():
                ID = result[y - 20:y + h + 5, x + w : x + w + 400]
                cv2_imshow(ID)  
                print("XIV")

 # getting total mark

        if "total" in b:
          if "of" not in pytesseract.image_to_string(result[y - 5 :y + h, x: x + w + 100],config='--psm 6').lower():
            if "mark" in pytesseract.image_to_string(result[y - 5 :y + h, x: x + w + 100],config='--psm 6').lower():
              if t == 3:
                print("\n")
                TotalMarks = result[y - 125 :y-20, x: x + w + 120]
                cv2_imshow(TotalMarks)
                print("XV")
              
              else:
                if t == 0:
                  print("\n")
                  TotalMarks = result[y - 35:y + h+ 25, x + w  : x + w + 250]
                  cv2_imshow(TotalMarks)
                  print("XVI")

            if t == 1: 
              print("\n")
              TotalMarks = result[y :y + h+ 75, x+w: x + w +170]
              cv2_imshow(TotalMarks)
              print("XVII")

            if t == 2:
              print("\n")
              TotalMarks = result[y - 60 :y + h+20, x + w: x + w +150]
              cv2_imshow(TotalMarks)
              print("XVIII")
            
            if t == 4:
              print("\n")
              TotalMarks = result[y - 45 :y + h + 10, x + w: x + w +150]
              cv2_imshow(TotalMarks)
              print("XIX")

            if t == 5: 
              print("\n")
              TotalMarks = result[y - 35 :y + h, x + w: x + w +150]
              cv2_imshow(TotalMarks)
              print("XX")

### predict and print output
      if t == 0:   
        print('\n\n================================ Full Name ================================ \n')
        cv2_imshow(SurName)
        try:
          SSN = predict_letter(SurName).lower()
        except UnboundLocalError:
          SFN = 'ERROR'
      else:

        print('\n\n================================ First Name ================================ \n')
        cv2_imshow(FirstName)
        try:
          SFN = predict_letter(FirstName).lower()
        except UnboundLocalError:
          SFN = 'ERROR'

      if t != 0:   
        print('\n\n================================ SurName ================================ \n')
        cv2_imshow(SurName)
        try:
          SSN = predict_letter(SurName).lower()
        except UnboundLocalError:
          SFN = 'ERROR'

      print('\n\n================================ ID ================================ \n')
      cv2_imshow(ID)
      if t == 5:
        try:
          SID = predict_digit(ID,0,0,30,0)
        except UnboundLocalError:
          SFN = 'ERROR'
      else:
        try:
          SID = predict_digit(ID,0,0,-40,0)
        except UnboundLocalError:
          SFN = 'ERROR'

      print('\n\n================================= TotalMarks ================================ \n')
      cv2_imshow(TotalMarks)
      try:
        STM = predict_digit(TotalMarks,0,0,0,0)
      except UnboundLocalError:
          SFN = 'ERROR'


      ###  BEFORE MATCHING
      print('\n\n##### BEFORE MATCHING #####')
      if t == 0:
        print("\nFirst Name: " + SSN)
      else:
        print("First Name: " + SFN)
        print("SurName: " + SSN)

      print("Student ID: " + SID)
      print("Total Marks: " + STM)

      ### AFTER MATCHING
      print('\n\n##### AFTER MATCHING #####')
      if t != 4:
        ID, FN, SN = matching(SID,SFN,SSN)
      else:
        ID, FN, SN =  matching(SID,'0',SSN) 

      print("Total Marks: " + STM)

      ### CREATE DATA FRAME

      df_new.loc[i]=[str(file), FN,SN, ID, STM]
      i =i+1
      my_list.clear()
      print("\n FINISH PROCESSING FILE: " + str(file))
      print("\n########################################################################################################################################\n")

  return df_new

In [3]:
model_save_name = 'digits.model'
path = F"/content/gdrive/My Drive/UC/S2/INFO601/{model_save_name}" 
model = keras.models.load_model(path)
print("Digit Model Loaded......")


model_save_name5 = 'model_hand1.h5'
path5 = F"/content/gdrive/My Drive/UC/S2/INFO601/{model_save_name5}" 
model5 = keras.models.load_model(path5)
print("Letters Model Loaded......")


print('# LOADING STUDENT INFO FROM DATA BASE')
path = F"/content/gdrive/My Drive/UC/S2/INFO601/NameID.xlsx"
df_raw = pd.read_excel(path, dtype=str)
df = df_raw
print("Databse Model Loaded......")
df.head(10)


link = "/content/gdrive/My Drive/UC/S2/INFO601/testdata/"
df_final = Main(link)

df_final

df_final.to_excel(F"/content/gdrive/My Drive/UC/S2/INFO601/Output.xlsx", index=False)
print("OUTPUT SAVED !")

print("\n\nFINISHED !")

##### END OF CODING

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df_final

,FILE NAME,FIRST NAME,LAST NAME,STUDENT ID,TOTAL MARKS
1319,test3.jpg,juliet,samandari,81955545,94
1303,test5.jpg,nicholas,blair,23068519,22
1392,test6.jpg,nicholas ning,lam,37848617,29
584,test50.jpg,alvin,connor,23517823,64
672,test10.jpg,niam,fox,97214358,37
631,test21.jpg,muir,tessa,98543219,36
534,test38.jpg,george,washington,43215679,33
632,20210131103346563_0009.jpg,anderson,cooper,34457156,20
533,20210131103346563_0026.jpg,diego,alcatraz,33251326,47
703,20210131103346563_0014.jpg,mateo,kahn,43256142,53
